In [1]:
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/resolve/main/source.spm
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/resolve/main/target.spm
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/raw/main/tokenizer_config.json
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/raw/main/vocab.json

--2022-05-22 07:08:28--  https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/resolve/main/source.spm
Resolving huggingface.co (huggingface.co)... 34.232.89.2, 18.214.24.217, 34.197.58.156, ...
Connecting to huggingface.co (huggingface.co)|34.232.89.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706917 (690K) [application/octet-stream]
Saving to: ‘source.spm’

source.spm          100%[===================>] 690.35K   528KB/s    in 1.3s    

2022-05-22 07:08:31 (528 KB/s) - ‘source.spm’ saved [706917/706917]

--2022-05-22 07:08:32--  https://huggingface.co/Helsinki-NLP/opus-mt-mul-en/resolve/main/target.spm
Resolving huggingface.co (huggingface.co)... 18.214.24.217, 34.197.58.156, 23.21.14.202, ...
Connecting to huggingface.co (huggingface.co)|18.214.24.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791194 (773K) [application/octet-stream]
Saving to: ‘target.spm’

target.spm          100%[===================>] 772.65K   553KB/s   

In [1]:
# !cp source.spm ep3/
# !cp target.spm ep3/
# !cp tokenizer_config.json ep3/
# !cp vocab.json ep3/

In [2]:
# !cp  -R outputs/checkpoint-149682-epoch-3/* ep3/

In [3]:
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

2022-05-22 15:49:47.851086: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
from tqdm.auto import tqdm

In [5]:
model_args = Seq2SeqArgs()

In [6]:
model_args

Seq2SeqArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=True, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_seed=None, max_grad_norm=1.0, max_seq

In [7]:
model = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="ep3/",
    args=model_args,
    use_cuda=True,
)

In [8]:
model.predict(['เล่า'])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['o']

In [9]:
import pandas as pd
import os

In [10]:
path_dataset = os.path.join('..','dataset','thai2rom')

train_filepaths = os.path.join(path_dataset,'train.tsv')
dev_filepaths = os.path.join(path_dataset,'dev.tsv')
test_filepaths = os.path.join(path_dataset,'test.tsv')

train_df = pd.read_csv(train_filepaths,sep="\t")
dev_df = pd.read_csv(dev_filepaths,sep="\t")
test_df = pd.read_csv(test_filepaths,sep="\t")

In [11]:
def load_data(data_path):
    new_df=pd.DataFrame()
    new_df["input_text"]=[str(i) for i in data_path['word']]
    new_df["target_text"] =[str(i) for i in data_path['roman']]
    return new_df


In [12]:
test_df = load_data(test_df)

In [13]:
list(test_df['input_text'])[0]

'กัลยา จันทร์มูล'

In [14]:
pred = model.predict(list(test_df['input_text']))

Generating outputs:   0%|          | 0/12477 [00:00<?, ?it/s]

In [15]:
pred[-1]

''

In [16]:
ground_truth = list(test_df['target_text'])

In [17]:
ground_truth[-1]

'hing'

In [18]:
pred[0]

'laya▁chanmun'

In [19]:
from jiwer import cer

In [20]:
def clean(s):
    if s.startswith('▁'):
        s=s.replace('▁','',1)
    return s.replace('▁',' ')

In [21]:
cer(ground_truth,[clean(i) for i in pred])

0.715839700651683

In [22]:
pred

['laya▁chanmun',
 'wi',
 'ison▁munphueng',
 'tthip laohawat',
 'luk',
 'da',
 'chan',
 'tri',
 'la',
 'chan',
 'in',
 'rueafan',
 'on',
 'thuk',
 't kitikoset',
 'trongnetpanya',
 'ngoen',
 'i',
 'non',
 'pru',
 'nit',
 'si',
 'kun',
 'ising chinphrawo',
 'nya',
 'k',
 'phila',
 'phup charoenphun',
 'eng',
 'kan',
 'som',
 'eng',
 'p',
 'kun',
 'wong',
 'at',
 't▁malaisirirat',
 'phan',
 '',
 'eo',
 'phum',
 'chan',
 'wet',
 'chon',
 'k',
 'klairung▁bunchok',
 'tprawat',
 'wong',
 'siang',
 'a',
 'ni',
 'si trangansi',
 'etnasaeng',
 'irung daenghiran',
 'i',
 'm',
 't',
 'hraikhriat',
 'eri',
 'phun',
 '',
 'p supphawanit',
 'pphlan',
 'eng',
 'na phuentana',
 'ngkaeo▁utsa',
 'bun',
 '',
 '',
 'wong',
 'ma',
 'phu',
 '',
 'ikaeo',
 'kun',
 'kon',
 'eng saeueng',
 'si',
 't',
 '',
 'ngsi',
 'tna prit',
 'i',
 'laet',
 'e',
 'm phuangmani',
 'ng',
 'klin',
 'eng',
 'ri',
 'chong',
 'raphon',
 'mimana',
 'ni',
 'yot',
 'oruchichinda',
 'ng',
 'ruea',
 'p',
 't',
 '',
 'cha',
 '',
 'kanon

In [23]:
with open("pred-ep4.txt",'w',encoding='utf-8') as f:
    f.write('\n'.join(pred))